In [2]:
import os
from data.source.pg_experiment import get_pg_experiment_dataframe
import polars as pl

from models.SimplifiedLightweightCNN import SimplifiedLightweightCNN
%load_ext autoreload
%autoreload 1
%aimport models.SimplifiedLightweightCNN
from models.SimpleCNN_v2 import train, evaluate
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from path import RESULT_DIRECTORY
import wandb

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
df_pron, df_tone = get_pg_experiment_dataframe(".ogg")
df_stageI_polish = df_pron.with_columns(word_id = pl.struct("word_id").rank("dense"))
df_stageI_polish = df_stageI_polish.filter(pl.col("word_id") == 1)
df_stageI_polish

get_pg_experiment_dataset(): WARNING, Dropped 2 rows with missing files
get_pg_experiment_dataset(): WARNING, Dropped 2 rows with missing files


<sys>:0: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
<sys>:0: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


id_student,value,word_id,rec_path,stage,univ,gender,mother
i64,i64,u32,str,i32,str,str,str
8,0,1,"""/home/tad1/Projects/AI/Mandari…",1,"""PG_CS_MA_1""","""m""","""polish"""
9,1,1,"""/home/tad1/Projects/AI/Mandari…",1,"""PG_CS_MA_1""","""f""","""polish"""
10,0,1,"""/home/tad1/Projects/AI/Mandari…",1,"""PG_CS_MA_1""","""m""","""polish"""
11,1,1,"""/home/tad1/Projects/AI/Mandari…",1,"""PG_CS_MA_1""","""f""","""polish"""
21,0,1,"""/home/tad1/Projects/AI/Mandari…",1,"""PG_CS_MA_1""","""f""","""polish"""
…,…,…,…,…,…,…,…
1537,1,1,"""/home/tad1/Projects/AI/Mandari…",1,"""CLES_UMK5""","""f""","""polish"""
1539,1,1,"""/home/tad1/Projects/AI/Mandari…",1,"""CLES_UMK5""","""m""","""polish"""
1542,1,1,"""/home/tad1/Projects/AI/Mandari…",1,"""CLES_UMK5""","""m""","""polish"""


In [4]:
import polars as pl
import numpy as np
from typing import Tuple

def stratified_split(df: pl.DataFrame, label_col: str, train_frac=0.8, val_frac=0.1, seed=42) -> Tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    classes = df.select(label_col).unique().to_series()
    train_rows, val_rows, test_rows = [], [], []

    rng = np.random.RandomState(seed)

    for cls in classes:
        class_df = df.filter(pl.col(label_col) == cls)
        n = class_df.height
        indices = rng.permutation(n)

        train_end = int(train_frac * n)
        val_end = int((train_frac + val_frac) * n)

        train_rows.append(class_df[indices[:train_end]])
        val_rows.append(class_df[indices[train_end:val_end]])
        test_rows.append(class_df[indices[val_end:]])

    train_df = pl.concat(train_rows)
    val_df = pl.concat(val_rows)
    test_df = pl.concat(test_rows)

    return train_df, val_df, test_df


In [20]:
from dataset import Cast, TorchDataset
from develop import reload_function, reload_module
import pytorch_dataloader
reload_module(pytorch_dataloader)
from pytorch_dataloader import ReshapeCollate, build_collate_fn, PaddingCollate, DefaultCollate
from functools import partial

from transformation import Channels, TorchVadLogMelSpec, TorchVadMFCC

reload_function(TorchVadMFCC)

TRAIN_SPLIT = 0.6
VAL_SPLIT = 0.2
TEST_SPLIT = 1 - TRAIN_SPLIT - VAL_SPLIT
train_pl, val_pl, test_pl = stratified_split(df_stageI_polish, label_col="value", train_frac=TRAIN_SPLIT, val_frac=VAL_SPLIT)

to_dataset = lambda dataframe: TorchDataset(
    Cast(dataframe.get_column("rec_path"), Channels("stack","multiply")(
            TorchVadMFCC(delta=0),
            TorchVadMFCC(delta=1),
            TorchVadMFCC(delta=2),
        )), 
    Cast(dataframe.get_column("value"), lambda x: torch.tensor(x).float()),
)

collate_fn = build_collate_fn(
    PaddingCollate(mode="SET_MAX_LEN", max_len=60, pad_dim=2),
    DefaultCollate(),
)
dataset_train = to_dataset(train_pl)
dataset_val = to_dataset(val_pl)
dataset_test = to_dataset(test_pl)

In [21]:
from pytorch_dataloader import MemoryLoadedDataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn, num_workers=4)
val_loader = DataLoader(dataset_val, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, collate_fn=collate_fn, num_workers=4)

input, label  = next(iter(train_loader))
print(input.shape, label.shape)

/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/761/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/761/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/761/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/668/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/668/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/668/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Manda

In [22]:
train_loader = MemoryLoadedDataLoader(train_loader, device=device)
print("Loaded train loader into memory")
val_loader = MemoryLoadedDataLoader(val_loader, device=device)
print("Loaded validation loader into memory")

/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/550/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/550/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/550/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/493/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/493/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Mandarin_Pronunciation_Recognition_Project/data/source/pg_dataset/recordings/stageI/493/a0.ogg has no speech segments, using full waveform
/home/tad1/Projects/AI/Manda

In [47]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(40* 60, 200),
    nn.ReLU(),
    nn.Dropout(0.8),
    nn.Linear(200, 50),
    nn.ReLU(),
    nn.Dropout(0.8),
    nn.Linear(50, 1),
    nn.Sigmoid()
)
model.eval()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=2400, out_features=200, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.8, inplace=False)
  (4): Linear(in_features=200, out_features=50, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.8, inplace=False)
  (7): Linear(in_features=50, out_features=1, bias=True)
  (8): Sigmoid()
)

In [38]:
str(model)

'Sequential(\n  (0): Flatten(start_dim=1, end_dim=-1)\n  (1): Linear(in_features=2400, out_features=100, bias=True)\n  (2): ReLU()\n  (3): Dropout(p=0.7, inplace=False)\n  (4): Linear(in_features=100, out_features=20, bias=True)\n  (5): ReLU()\n  (6): Dropout(p=0.5, inplace=False)\n  (7): Linear(in_features=20, out_features=1, bias=True)\n  (8): Sigmoid()\n)'

In [23]:
from models.SimpleCNN_v2 import SimpleCNN
reload_function(SimpleCNN)
model = SimpleCNN()

In [ ]:
# Model variables definition.
pth = "SimplifiedLightweightCNN.pth"
lr = 4e-5  # Reduce from 1e-3
epochs = 100
model = model.to(device)
reload_function(train)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=2e-4)  # Add L2 regularization
# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)
criterion = nn.BCELoss()

# Start a new wandb run to track this script.
run = wandb.init(
    # name of the run
    name="Hypothesis-5 SimpleCNN",
    config={
        "Name": 'SimplifiedLightweightCNN',
        "learning_rate": lr,
        "optimizer": "Adam",
        "criterion": "BCELoss",
        "architecture": "SimplifiedLightweightCNN",
        "architecture_details": str(model),
        "dataset": "Stage-I-only-polish",
        "train_val_test(%)": f'{TRAIN_SPLIT}-{VAL_SPLIT}-{TEST_SPLIT}',
        "epochs": epochs,
    },
)

# Training loop
for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    # Update learning rate
    scheduler.step(val_loss)
    # Logging the metadata for each epoch so that the charts can be generated on the dashboard
    run.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, })
    print(
        f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

run.log({"model_eval": model.eval()})
# Saving the model to pth and adding it to the artifacts of the run, there is 5GB of memory on wandb, so we should be fine.
torch.save(model.state_dict(), os.path.join(RESULT_DIRECTORY, pth))
artifact = wandb.Artifact("SimplifiedLightweightCNN-model", type="model")
artifact.add_file(os.path.join(RESULT_DIRECTORY, pth))
run.log_artifact(artifact)

# Finish the run so it gets sent to the remote. You can discover the run right after that on the dashboard.
run.finish()


Epoch 1, Train Loss: 0.6823, Train Acc: 0.5839, Val Loss: 0.6775, Val Acc: 0.5922
Epoch 2, Train Loss: 0.6785, Train Acc: 0.5677, Val Loss: 0.6674, Val Acc: 0.5922
Epoch 3, Train Loss: 0.6818, Train Acc: 0.5839, Val Loss: 0.6655, Val Acc: 0.5922
Epoch 4, Train Loss: 0.6575, Train Acc: 0.5968, Val Loss: 0.6619, Val Acc: 0.5922
Epoch 5, Train Loss: 0.6653, Train Acc: 0.5806, Val Loss: 0.6615, Val Acc: 0.6019
Epoch 6, Train Loss: 0.6527, Train Acc: 0.6290, Val Loss: 0.6624, Val Acc: 0.6214
Epoch 7, Train Loss: 0.6763, Train Acc: 0.6194, Val Loss: 0.6606, Val Acc: 0.6019
Epoch 8, Train Loss: 0.6412, Train Acc: 0.6419, Val Loss: 0.6602, Val Acc: 0.6311
Epoch 9, Train Loss: 0.6523, Train Acc: 0.6097, Val Loss: 0.6565, Val Acc: 0.6408
Epoch 10, Train Loss: 0.6365, Train Acc: 0.6355, Val Loss: 0.6524, Val Acc: 0.6117
Epoch 11, Train Loss: 0.6404, Train Acc: 0.6355, Val Loss: 0.6571, Val Acc: 0.6214
Epoch 12, Train Loss: 0.6234, Train Acc: 0.6387, Val Loss: 0.6494, Val Acc: 0.6505
Epoch 13, Tra

train_acc,▁▁▂▂▂▄▃▃▅▄▄▅▄▅▆▆▆▇▆▇▇▇▇▇█▇▇▇▇█▇██▇▇▇▇▇▇▆
train_loss,██▇▇▇▇▆▆▆▆▅▅▄▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▂▂▂▁▂▁▂▂▂▂▁▂
val_acc,▁▁▂▃▃▅▆▆▄▅▅▇▅▆▅▇▆█▆▇▆▆▇█▆▇▇█▇▇█▇▇█▇▇▇▇▇█
val_loss,█▇▆▆▆▅▃▃▃▄▃▃▃▁▄▁▃▃▄▁▃▃▃▄▃▄▄▄▄▄▄▅▄▅▅▄▅▄▄▄
model_eval,SimpleCNN( (conv1)...
train_acc,0.8
train_loss,0.43852
val_acc,0.67961
val_loss,0.65297
